# Contexte Problématique 
Le but de ce projet est d'étudier l'influence des publications Reddit sur la tendace des prix des crypto-monnaies. Pour ce faire, nous avons eu recours à une base de données composée de 5 tables (close, high, low, open, volume, reddit)

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import shapiro
from scipy.stats import kstest
from scipy.stats import norm


# Loading Data

In [2]:
close_df=pd.read_csv("/kaggle/input/crypto/close.csv")
high_df=pd.read_csv("/kaggle/input/crypto/high.csv")
low_df=pd.read_csv("/kaggle/input/crypto/low.csv")
open_df=pd.read_csv("/kaggle/input/crypto/open.csv")
volume_df=pd.read_csv("/kaggle/input/crypto/volume.csv")

# Data exploration

In [3]:
open_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10992 entries, 0 to 10991
Columns: 128 entries, Unnamed: 0 to ZRXUSDT
dtypes: float64(127), object(1)
memory usage: 10.7+ MB


In [4]:
open_df.describe()

,1000SHIBUSDT,1000XECUSDT,1INCHUSDT,AAVEUSDT,ADAUSDT,ALGOUSDT,ALICEUSDT,ALPHAUSDT,ANKRUSDT,ANTUSDT,...,XEMUSDT,XLMUSDT,XMRUSDT,XRPUSDT,XTZUSDT,YFIUSDT,ZECUSDT,ZENUSDT,ZILUSDT,ZRXUSDT
count,10992.000000,8396.000000,10992.000000,10992.000000,10992.000000,10992.000000,10992.000000,10992.000000,10992.000000,5971.000000,...,10992.000000,10992.000000,10992.000000,10992.000000,10992.000000,10992.000000,10992.000000,10992.000000,10992.000000,10992.000000
mean,0.019872,0.098871,2.102563,209.620022,1.263301,0.997991,8.400483,0.551161,0.075847,4.006560,...,0.118135,0.236829,208.330772,0.741749,3.595072,23993.123735,124.448204,52.384709,0.073593,0.714977
std,0.013732,0.059343,1.109594,101.648447,0.652843,0.542270,5.160177,0.355698,0.030511,2.675575,...,0.052556,0.089883,49.870137,0.276730,1.654891,11079.156475,41.894544,27.798080,0.029981,0.286056
min,0.004948,0.028040,0.521700,46.130000,0.406900,0.280000,1.844000,0.095400,0.023300,1.180000,...,0.035700,0.099220,99.390000,0.294300,1.210000,4103.000000,50.670000,12.462000,0.029890,0.225400
25%,0.008392,0.046810,1.074500,115.060000,0.655400,0.583050,3.555500,0.221000,0.050257,1.745000,...,0.062575,0.149960,167.090000,0.514900,2.173000,13986.500000,94.787500,25.884500,0.045640,0.471350
50%,0.013131,0.081835,2.007850,190.245000,1.193050,0.886850,6.969000,0.458700,0.077080,3.587000,...,0.116200,0.236735,207.185000,0.761500,3.251000,24857.000000,123.785000,50.507500,0.070675,0.722650
75%,0.027224,0.117855,2.919750,299.410000,1.656225,1.438525,12.583750,0.861725,0.096850,5.283500,...,0.166200,0.316293,250.990000,0.942975,4.488250,33347.500000,152.792500,74.794750,0.098863,0.951400
max,0.086259,0.269880,6.712600,445.740000,3.099400,2.440600,28.494000,1.495100,0.206090,13.866000,...,0.243600,0.433840,339.000000,1.413700,8.987000,47365.700000,302.500000,133.926000,0.218280,1.513800


In [5]:
open_df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Unnamed: 0,1000SHIBUSDT,1000XECUSDT,1INCHUSDT,AAVEUSDT,ADAUSDT,ALGOUSDT,ALICEUSDT,ALPHAUSDT,ANKRUSDT,...,XEMUSDT,XLMUSDT,XMRUSDT,XRPUSDT,XTZUSDT,YFIUSDT,ZECUSDT,ZENUSDT,ZILUSDT,ZRXUSDT
0,2021-06-01 00:00:00,0.009403,NaN,3.1777,376.52,1.7002,0.9164,7.226,0.8469,0.10439,...,0.1889,0.39402,272.01,1.0111,3.542,46455.4,158.15,107.525,0.11350,1.0054
1,2021-06-01 01:00:00,0.009278,NaN,3.2128,383.13,1.7402,0.9369,7.431,0.8676,0.10638,...,0.1951,0.40272,272.91,1.0459,3.606,46973.5,162.25,107.785,0.11551,1.0210
2,2021-06-01 02:00:00,0.009193,NaN,3.2544,385.81,1.7719,0.9399,7.599,0.8827,0.10847,...,0.1963,0.41357,273.26,1.0905,3.605,47365.7,163.18,111.316,0.11729,1.0258
3,2021-06-01 03:00:00,0.009095,NaN,3.1696,377.43,1.7275,0.9185,7.310,0.8548,0.10668,...,0.1922,0.40129,267.61,1.0601,3.541,46123.1,158.51,108.772,0.11468,0.9961
4,2021-06-01 04:00:00,0.009043,NaN,3.1456,382.25,1.7113,0.9164,7.218,0.8509,0.10767,...,0.1916,0.40234,270.87,1.0445,3.541,45830.4,157.07,107.613,0.11396,1.0019


In [6]:
open_df.tail()

,Unnamed: 0,1000SHIBUSDT,1000XECUSDT,1INCHUSDT,AAVEUSDT,ADAUSDT,ALGOUSDT,ALICEUSDT,ALPHAUSDT,ANKRUSDT,...,XEMUSDT,XLMUSDT,XMRUSDT,XRPUSDT,XTZUSDT,YFIUSDT,ZECUSDT,ZENUSDT,ZILUSDT,ZRXUSDT
10987,2022-09-01 19:00:00,0.011997,0.04522,0.6502,84.09,0.4498,0.2882,1.986,0.1195,0.03401,...,0.0427,0.10290,151.55,0.3245,1.498,8993.0,57.67,15.506,0.03549,0.2909
10988,2022-09-01 20:00:00,0.012051,0.04531,0.6533,84.47,0.4524,0.2880,1.993,0.1199,0.03409,...,0.0429,0.10322,151.48,0.3270,1.500,8994.0,57.87,15.519,0.03560,0.2917
10989,2022-09-01 21:00:00,0.012082,0.04571,0.6566,85.29,0.4528,0.2895,2.009,0.1204,0.03439,...,0.0429,0.10385,151.68,0.3295,1.507,9032.0,58.20,15.606,0.03566,0.2927
10990,2022-09-01 22:00:00,0.012233,0.04596,0.6631,86.00,0.4561,0.2918,2.030,0.1216,0.03475,...,0.0435,0.10427,152.87,0.3320,1.522,9173.0,59.13,15.701,0.03596,0.2953
10991,2022-09-01 23:00:00,0.012271,0.04605,0.6651,86.52,0.4567,0.2916,2.035,0.1220,0.03483,...,0.0437,0.10453,153.94,0.3318,1.529,9274.0,59.28,15.712,0.03616,0.2954


In [7]:
# renaming the first column in each data frame 'date'

for df in [close_df, high_df, low_df, open_df, volume_df]:
    df.rename(columns={'Unnamed: 0':'date'}, inplace=True)

In [8]:
# Converts the 'date' column in the DataFrame 'open_df' to datetime type 
open_df['date'] = pd.to_datetime(open_df['date'])

In [9]:
# checking the missing values 

missing_values = high_df.isna().sum()
for column, count in missing_values .items():
        print(f"{column}: {count}")

date: 0
1000SHIBUSDT: 0
1000XECUSDT: 2596
1INCHUSDT: 0
AAVEUSDT: 0
ADAUSDT: 0
ALGOUSDT: 0
ALICEUSDT: 0
ALPHAUSDT: 0
ANKRUSDT: 0
ANTUSDT: 5021
ARPAUSDT: 3364
ARUSDT: 2860
ATAUSDT: 2188
ATOMUSDT: 0
AUDIOUSDT: 1900
AVAXUSDT: 0
AXSUSDT: 0
BAKEUSDT: 0
BALUSDT: 0
BANDUSDT: 0
BATUSDT: 0
BCHUSDT: 0
BELUSDT: 0
BLZUSDT: 0
BNBUSDT: 0
BTCDOMUSDT: 484
BTCUSDT: 0
C98USDT: 2020
CELOUSDT: 2836
CELRUSDT: 0
CHRUSDT: 0
CHZUSDT: 0
COMPUSDT: 0
COTIUSDT: 0
CRVUSDT: 0
CTKUSDT: 0
CTSIUSDT: 3532
CVCUSDT: 0
DASHUSDT: 0
DEFIUSDT: 0
DENTUSDT: 0
DGBUSDT: 0
DOGEUSDT: 0
DOTUSDT: 0
DUSKUSDT: 5284
DYDXUSDT: 2428
EGLDUSDT: 0
ENJUSDT: 0
ENSUSDT: 4372
EOSUSDT: 0
ETCUSDT: 0
ETHUSDT: 0
FILUSDT: 0
FLMUSDT: 0
FTMUSDT: 0
GALAUSDT: 2620
GRTUSDT: 0
GTCUSDT: 244
HBARUSDT: 0
HNTUSDT: 0
HOTUSDT: 0
ICPUSDT: 10992
ICXUSDT: 0
IOSTUSDT: 0
IOTAUSDT: 0
IOTXUSDT: 1732
KAVAUSDT: 0
KLAYUSDT: 3196
KNCUSDT: 0
KSMUSDT: 0
LINAUSDT: 0
LINKUSDT: 0
LITUSDT: 0
LPTUSDT: 3916
LRCUSDT: 0
LTCUSDT: 0
MANAUSDT: 0
MASKUSDT: 2092
MATICUSDT: 0
MKRUSDT: 0
M

In [10]:
# the columns with missing values

missing_values = high_df.isna().sum()
for column, count in missing_values .items():
    if count>0:
        print(f"{column}: {count}")

1000XECUSDT: 2596
ANTUSDT: 5021
ARPAUSDT: 3364
ARUSDT: 2860
ATAUSDT: 2188
AUDIOUSDT: 1900
BTCDOMUSDT: 484
C98USDT: 2020
CELOUSDT: 2836
CTSIUSDT: 3532
DUSKUSDT: 5284
DYDXUSDT: 2428
ENSUSDT: 4372
GALAUSDT: 2620
GTCUSDT: 244
ICPUSDT: 10992
IOTXUSDT: 1732
KLAYUSDT: 3196
LPTUSDT: 3916
MASKUSDT: 2092
PEOPLEUSDT: 4949
RAYUSDT: 1924
ROSEUSDT: 5116


In [11]:
# replacing missing values with 0

close_df.fillna(0, inplace=True)
high_df.fillna(0, inplace=True)
low_df.fillna(0, inplace=True)
open_df.fillna(0, inplace=True)
volume_df.fillna(0, inplace=True)

# Selection of a coin: 'ETHUSDT'

In [12]:
# creation of a single data table with the columns high/low/open/close

eth_df = pd.concat([open_df['date'], open_df['ETHUSDT'], close_df['ETHUSDT'], high_df['ETHUSDT'], low_df['ETHUSDT']], axis=1)
eth_df.columns = ['date', 'open', 'close', 'high', 'low']
print(eth_df)

                     date     open    close     high      low
0     2021-06-01 00:00:00  2647.30  2714.39  2721.51  2638.44
1     2021-06-01 01:00:00  2714.38  2732.20  2741.45  2689.00
2     2021-06-01 02:00:00  2732.27  2672.02  2732.71  2662.00
3     2021-06-01 03:00:00  2672.64  2643.57  2676.25  2613.15
4     2021-06-01 04:00:00  2643.09  2631.54  2678.00  2612.77
...                   ...      ...      ...      ...      ...
10987 2022-09-01 19:00:00  1555.57  1559.46  1572.00  1552.00
10988 2022-09-01 20:00:00  1559.47  1563.07  1569.30  1555.71
10989 2022-09-01 21:00:00  1563.07  1579.69  1584.94  1561.02
10990 2022-09-01 22:00:00  1579.69  1581.83  1595.00  1576.09
10991 2022-09-01 23:00:00  1581.82  1584.84  1599.00  1580.58

[10992 rows x 5 columns]


In [13]:
#Calcul du rendement et l'ajouter comme une nouvelle colonne

eth_df['rendement'] = eth_df['close'] / eth_df['open']
print(eth_df)

                     date     open    close     high      low  rendement
0     2021-06-01 00:00:00  2647.30  2714.39  2721.51  2638.44   1.025343
1     2021-06-01 01:00:00  2714.38  2732.20  2741.45  2689.00   1.006565
2     2021-06-01 02:00:00  2732.27  2672.02  2732.71  2662.00   0.977949
3     2021-06-01 03:00:00  2672.64  2643.57  2676.25  2613.15   0.989123
4     2021-06-01 04:00:00  2643.09  2631.54  2678.00  2612.77   0.995630
...                   ...      ...      ...      ...      ...        ...
10987 2022-09-01 19:00:00  1555.57  1559.46  1572.00  1552.00   1.002501
10988 2022-09-01 20:00:00  1559.47  1563.07  1569.30  1555.71   1.002308
10989 2022-09-01 21:00:00  1563.07  1579.69  1584.94  1561.02   1.010633
10990 2022-09-01 22:00:00  1579.69  1581.83  1595.00  1576.09   1.001355
10991 2022-09-01 23:00:00  1581.82  1584.84  1599.00  1580.58   1.001909

[10992 rows x 6 columns]


# Normalization of the 'rendement' column

In [14]:
from sklearn.preprocessing import QuantileTransformer
# Créez une instance de QuantileTransformer
quantile_transformer = QuantileTransformer( n_quantiles=4000, output_distribution='normal', ignore_implicit_zeros=False, subsample=10000, random_state=None, copy=False)
returns_2d = eth_df['rendement'].values.reshape(-1, 1)
# Ajustez et transformez les rendements
transformed_returns = quantile_transformer.fit_transform(returns_2d)
print(eth_df)

                     date     open    close     high      low  rendement
0     2021-06-01 00:00:00  2647.30  2714.39  2721.51  2638.44   2.251404
1     2021-06-01 01:00:00  2714.38  2732.20  2741.45  2689.00   0.954135
2     2021-06-01 02:00:00  2732.27  2672.02  2732.71  2662.00  -2.061496
3     2021-06-01 03:00:00  2672.64  2643.57  2676.25  2613.15  -1.363796
4     2021-06-01 04:00:00  2643.09  2631.54  2678.00  2612.77  -0.686013
...                   ...      ...      ...      ...      ...        ...
10987 2022-09-01 19:00:00  1555.57  1559.46  1572.00  1552.00   0.392851
10988 2022-09-01 20:00:00  1559.47  1563.07  1569.30  1555.71   0.363570
10989 2022-09-01 21:00:00  1563.07  1579.69  1584.94  1561.02   1.372121
10990 2022-09-01 22:00:00  1579.69  1581.83  1595.00  1576.09   0.206796
10991 2022-09-01 23:00:00  1581.82  1584.84  1599.00  1580.58   0.301536

[10992 rows x 6 columns]


In [15]:
eth_df['date'] = pd.to_datetime(eth_df['date'])

In [16]:
reddit_df=pd.read_csv("/kaggle/input/crypto/reddit_cc.csv")

In [17]:
reddit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40918 entries, 0 to 40917
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      40918 non-null  object 
 1   score      40918 non-null  int64  
 2   id         40918 non-null  object 
 3   url        18774 non-null  object 
 4   comms_num  40918 non-null  int64  
 5   created    40918 non-null  float64
 6   body       33733 non-null  object 
 7   timestamp  40918 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 2.5+ MB


In [18]:
reddit_df.describe()

,score,comms_num,created
count,40918.000000,40918.000000,4.091800e+04
mean,16.115377,31.327778,1.630084e+09
std,200.299711,427.511306,8.572618e+05
min,-5.000000,0.000000,1.628887e+09
25%,1.000000,0.000000,1.629268e+09
50%,2.000000,0.000000,1.630103e+09
75%,6.000000,25.000000,1.630748e+09
max,16226.000000,35579.000000,1.632042e+09


In [19]:
reddit_df.head()

,title,score,id,url,comms_num,created,body,timestamp
0,RSUN is in a parabolic rise. The FOMO is real....,0,p4blrv,https://www.reddit.com/r/CryptoCurrency/commen...,18,1.628960e+09,"NFT’s, more specifically NFT card games are ta...",2021-08-14 19:51:50
1,If you see a good post don’t forget to tip the...,2,p4bld3,https://www.reddit.com/r/CryptoCurrency/commen...,37,1.628960e+09,"For people who don’t already know, if you like...",2021-08-14 19:51:10
2,I hope moons fail.,0,p4bl77,https://www.reddit.com/r/CryptoCurrency/commen...,61,1.628960e+09,Everyone in this sub is obsessed with moons. E...,2021-08-14 19:50:55
3,If you could get 1 BTC or 20 ETH or 50000 ADA ...,12,p4bfmn,https://www.reddit.com/r/CryptoCurrency/commen...,126,1.628959e+09,Imagine you get the chance to choose one of th...,2021-08-14 19:42:23
4,My husband (investment banker) doesn’t trust m...,0,p4bb2i,https://www.reddit.com/r/CryptoCurrency/commen...,27,1.628959e+09,\nI love my daughter and want the best for her...,2021-08-14 19:35:17


In [20]:
reddit_df= reddit_df[['score','timestamp']]

In [21]:
reddit_df['timestamp'] = pd.to_datetime(reddit_df['timestamp'])

# Sort the DataFrame by timestamp
reddit_df.sort_values(by='timestamp', inplace=True)

# Calculate rolling averages for different periods
reddit_df['Score_1h'] = reddit_df['score'].rolling(window=1).mean()
reddit_df['Score_3h'] = reddit_df['score'].rolling(window=3).mean()
reddit_df['Score_8h'] = reddit_df['score'].rolling(window=8).mean()
reddit_df['Score_12h'] = reddit_df['score'].rolling(window=12).mean()
reddit_df['Score_24h'] = reddit_df['score'].rolling(window=24).mean()

In [22]:
reddit_df=reddit_df.dropna()

In [23]:
# Round the timestamp to the next hour
reddit_df['timestamp'] = pd.to_datetime(reddit_df['timestamp'])

reddit_df = reddit_df.groupby([reddit_df['timestamp'].dt.date, reddit_df['timestamp'].dt.hour]).mean()

reddit_df['timestamp']=(reddit_df['timestamp'] + pd.Timedelta('1 hour')).dt.floor('H')

reddit_df = reddit_df.reset_index(drop=True)


/tmp/ipykernel_36/2197189700.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  reddit_df['timestamp']=(reddit_df['timestamp'] + pd.Timedelta('1 hour')).dt.floor('H')


In [24]:
# Rename the column 'timestamp' as 'date'
reddit_df = reddit_df.rename(columns={'timestamp': 'date'})

In [25]:
reddit_df

,score,date,Score_1h,Score_3h,Score_8h,Score_12h,Score_24h
0,14.000000,2021-08-14 00:00:00,14.000000,13.592593,18.222222,19.953704,33.870370
1,15.844828,2021-08-14 01:00:00,15.844828,15.844828,15.849138,15.241379,14.804598
2,10.893617,2021-08-14 02:00:00,10.893617,10.893617,10.957447,12.108156,14.992908
3,8.250000,2021-08-14 03:00:00,8.250000,8.136364,8.011364,8.168561,8.482008
4,24.803279,2021-08-14 04:00:00,24.803279,24.797814,24.752049,24.766393,24.605874
...,...,...,...,...,...,...,...
548,27.312500,2021-09-19 09:00:00,27.312500,28.187500,29.804688,30.083333,36.880208
549,16.944444,2021-09-19 10:00:00,16.944444,16.574074,14.645833,14.888889,21.071759
550,8.454545,2021-09-19 11:00:00,8.454545,9.424242,11.943182,11.568182,12.196970
551,1.303067,2021-09-19 12:00:00,1.303067,1.304294,1.330521,1.352556,1.426943


In [26]:
# Fusion des DataFrames eth_df et reddit_df sur la colonne 'date'
df= pd.merge(eth_df, reddit_df, on='date', how='inner')
df

,date,open,close,high,low,rendement,score,Score_1h,Score_3h,Score_8h,Score_12h,Score_24h
0,2021-08-14 00:00:00,3284.32,3324.38,3332.88,3284.32,1.500517,14.000000,14.000000,13.592593,18.222222,19.953704,33.870370
1,2021-08-14 01:00:00,3324.38,3309.98,3328.68,3287.08,-0.681954,15.844828,15.844828,15.844828,15.849138,15.241379,14.804598
2,2021-08-14 02:00:00,3309.98,3307.00,3325.65,3301.13,-0.148769,10.893617,10.893617,10.893617,10.957447,12.108156,14.992908
3,2021-08-14 03:00:00,3306.99,3307.00,3312.22,3285.23,-0.000965,8.250000,8.250000,8.136364,8.011364,8.168561,8.482008
4,2021-08-14 04:00:00,3307.01,3298.89,3313.00,3290.83,-0.388918,24.803279,24.803279,24.797814,24.752049,24.766393,24.605874
...,...,...,...,...,...,...,...,...,...,...,...,...
548,2021-09-19 09:00:00,3436.66,3442.99,3453.00,3436.66,0.289805,27.312500,27.312500,28.187500,29.804688,30.083333,36.880208
549,2021-09-19 10:00:00,3442.99,3427.79,3449.88,3417.95,-0.692053,16.944444,16.944444,16.574074,14.645833,14.888889,21.071759
550,2021-09-19 11:00:00,3427.79,3410.57,3430.00,3407.49,-0.771465,8.454545,8.454545,9.424242,11.943182,11.568182,12.196970
551,2021-09-19 12:00:00,3410.57,3327.07,3410.57,3301.00,-2.174594,1.303067,1.303067,1.304294,1.330521,1.352556,1.426943
